## This is a notebook that verify the communication of the pointing kernel with M2 (using the rotator)

In this notebook, we 
* point the telescope to a particular pointing, 
* make sure the mt telemetry is correct
* make sure that the zenith angle is picked up by M2 via the subscription
* check that M2 LUT forces vary with the zenith angle acccordingly


In [ ]:
#from lsst.ts.observatory.maintel import MTCS
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

import asyncio

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime
import pandas as pd

In [ ]:
import os
print(os.environ["OSPL_URI"])

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
#await asyncio.sleep(10)

In [ ]:
ptg = salobj.Remote(script.domain, "MTPtg")
rot = salobj.Remote(script.domain, "MTRotator")
m2 = salobj.Remote(script.domain, "MTM2")
mount = salobj.Remote(script.domain, "MTMount")
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await asyncio.gather(ptg.start_task,
                     mount.start_task,
                     script.start_task, #                     
                     rot.start_task,
                     m2.start_task)

In [ ]:
h = await ptg.evt_heartbeat.next(flush=True, timeout=5)
print(h.heartbeat)

In [ ]:
#As long as you get something for the payload its OK.
h = await m2.evt_heartbeat.next(flush=True, timeout=5)
print(h.heartbeat)

In [ ]:
state = await ptg.evt_summaryState.aget(timeout=5)
print('ptg', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
state = await m2.evt_summaryState.aget(timeout=5)
print('m2',salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
state = await rot.evt_summaryState.aget(timeout=5)
print('rot', salobj.State(state.summaryState),pd.to_datetime(state.private_sndStamp, unit='s'))
#state = await mount.evt_summaryState.aget(timeout=5)
#print('mount', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))

In [ ]:
#await salobj.set_summary_state(m2, salobj.State.DISABLED)
await salobj.set_summary_state(m2, salobj.State.ENABLED)

In [ ]:
#if ptg is in standby (5), enable it
await salobj.set_summary_state(ptg, salobj.State.ENABLED)

Check the state of the system

In [ ]:
axialForces = await m2.tel_axialForce.next(flush=True, timeout=2)
tangentForces = await m2.tel_tangentForce.next(flush=True, timeout=2)

In [ ]:
m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
print("starting with Status of the M2 force balance system ---", m2ForceBalance.status, "----",
      pd.to_datetime(m2ForceBalance.private_sndStamp, unit='s'))
if not m2ForceBalance.status:
    await m2.cmd_switchForceBalanceSystem.set_start(status=True, timeout=10)
    m2ForceBalance = await m2.evt_forceBalanceSystemStatus.aget(timeout=10.)
    print("Status of the M2 force balance system", m2ForceBalance.status)

### Enter the location for the telescope for the pointing

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

## Start of a slew

In [ ]:
now = datetime.now()
print("Start to point the telescope", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

In [ ]:
target_name="TMA motion test"
time_data = await ptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_ptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_ptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

print(curr_time_ptg.tai.value)

cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_ptg.tai, 
                location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
# Calculating the other parameters     
rot_pa = rot_tel
await rot.cmd_trackStart.start(timeout=30.)


In [ ]:
#The pointing component is commanding the mount directly
ack = await ptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)
print("System Ready")

## Check telemetry: list here what we want to check once the slew is done

In [ ]:
# Check that the mirror is in Position --  this check is a little more tricky if we are in closed loop. 
m2InPosition = await m2.evt_m2AssemblyInPosition.aget(timeout=10.)
print("Is the m2 in Position after a slew?", m2InPosition.inPosition)

In [ ]:
from lsst.ts.idl.enums import MTM2

In [ ]:
m2AngleSource = await m2.evt_inclinationTelemetrySource.aget(timeout=10.)
print("Inclinometer Source", MTM2.InclinationTelemetrySource(m2AngleSource.source))

#zenithAngleData is old and observing
m2Angle = await m2.tel_zenithAngle.aget(timeout=10.)
print("elevation from the source", 90 - m2Angle.measured)

mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual)

## Stop the system: stop tracking, what else?

In [ ]:
await ptg.cmd_stopTracking.start(timeout=10.)